In [12]:
from math import ceil, log
from sage.all import GF, gcd, inverse_mod  # Assuming you are using SageMath

def find_exponent(p):
    d = 2
    while gcd(d, p-1) > 1:
        d += 1
    return d

class miniMiMC:
    def __init__(self, field_size, number_of_rounds=None, optional_x=None):
        # Find the exponent and then reduce it modulo (field_size-1)
        raw_exp = find_exponent(field_size)
        self.exponent = raw_exp % (field_size - 1)
        if self.exponent == 0:
            self.exponent = field_size - 1
            
        if number_of_rounds is None:
            self.number_of_rounds = ceil(log(field_size, self.exponent))
        else:       
            self.number_of_rounds = number_of_rounds
        
        self.field = GF(field_size)
        self.field_size = field_size
        # Define a polynomial ring over GF(field_size)
        self.R = self.field['X']
        self.X = self.R.gen()
        # Define the reduction polynomial to get a unique representation
        # of a function from GF(q) to GF(q) with degree < field_size.
        self.reduction_poly = self.X**field_size - self.X

    def round_function(self, x, ki):
        # Apply the round function and then reduce the degree modulo X^q - X
        exp = self.exponent % (self.field_size - 1)
        result = (x + ki) ** exp
        result = result % self.reduction_poly  # Degree reduction here
        return result

    def encrypt(self, plaintext, master_key):
        x = plaintext
        for i in range(self.number_of_rounds):
            x = self.round_function(x, master_key + i)
        return x

    def decrypt(self, ciphertext, master_key):
        x = ciphertext 
        inv_exp = inverse_mod(self.exponent, self.field_size - 1)
        for i in reversed(range(self.number_of_rounds)):
            x = (x ** inv_exp) + (master_key - i)
            x = x % self.reduction_poly  # Ensure reduction in decryption too
        return x

In [13]:
def find_zero_sum_coeffs_first(pol):
    coeff_dict = pol.dict()  # get polynomial coefficients as {exponent: coefficient}
    zero_sum_coeffs = []

    # Convert coefficient dictionary to list of (exponent, coefficient)
    coeff_items = list(coeff_dict.items())

    # Check all subsets of coefficients for sums equal to zero
    from itertools import combinations

    # Iterate through subsets of different lengths
    for r in range(2, len(coeff_items) + 1):
        for subset in combinations(coeff_items, r):
            coeff_sum = sum(coefficient for _, coefficient in subset)
            if coeff_sum == 0:
                return subset
                


def find_zero_sum_coeffs(pol):
    coeff_dict = pol.dict()  # get polynomial coefficients as {exponent: coefficient}
    zero_sum_coeffs = []

    # Convert coefficient dictionary to list of (exponent, coefficient)
    coeff_items = list(coeff_dict.items())

    # Check all subsets of coefficients for sums equal to zero
    from itertools import combinations

    # Iterate through subsets of different lengths
    for r in range(2, len(coeff_items) + 1):
        for subset in combinations(coeff_items, r):
            coeff_sum = sum(coefficient for _, coefficient in subset)
            if coeff_sum == 0:
                zero_sum_coeffs.append(subset)
                break

    return zero_sum_coeffs


def return_first_missing_monomial(pol):
    for i in range (pol.degree()):
        coeff = pol[i]
    
    # Check presence based on whether coefficient is non-zero
        is_present = coeff != 0
        if is_present == False:
            print(f"X^{i} is NOT present: {is_present}, coefficient: {coeff}")
            

In [14]:
q = 2**12-5
cipher = miniMiMC(q, 7)

gf = GF(q)
X = gf.polynomial_ring("X").gen()
for _ in range(10):
    K= gf.random_element()
    print(K)
    pol = cipher.encrypt(X, K)
    return_first_missing_monomial(pol)

3048
2849
X^1234 is NOT present: False, coefficient: 0
3547
X^287 is NOT present: False, coefficient: 0
X^1204 is NOT present: False, coefficient: 0
3287
1513
X^1343 is NOT present: False, coefficient: 0
234
1279
238
3640
X^1125 is NOT present: False, coefficient: 0
327


In [15]:
q = 71
cipher = miniMiMC(q, 4)
gf = GF(q)

for _ in range(1000):
    K = gf.random_element()
    print("key:", K)
    X = gf.polynomial_ring("X").gen()
    pol = cipher.encrypt(X, K)
    return_first_missing_monomial(pol)
    print(pol)
#     print(find_zero_sum_coeffs_first(pol))

key: 33
X^1 is NOT present: False, coefficient: 0
X^18 is NOT present: False, coefficient: 0
37*X^69 + 43*X^68 + 37*X^67 + 61*X^66 + 50*X^65 + 34*X^64 + 53*X^63 + 7*X^62 + 67*X^61 + 28*X^60 + 29*X^59 + 36*X^58 + 28*X^57 + 22*X^56 + 58*X^55 + 62*X^54 + 13*X^53 + 25*X^52 + 57*X^51 + 45*X^50 + 58*X^49 + 24*X^48 + 28*X^47 + 66*X^46 + 54*X^45 + 41*X^44 + 28*X^43 + 28*X^42 + 52*X^41 + 9*X^40 + 39*X^39 + 36*X^38 + X^37 + 20*X^36 + 7*X^35 + 64*X^34 + 56*X^33 + 33*X^32 + 45*X^31 + 35*X^30 + 29*X^29 + 11*X^28 + 67*X^27 + 30*X^26 + 20*X^25 + 48*X^24 + 15*X^23 + 58*X^22 + 19*X^21 + 15*X^20 + 33*X^19 + 26*X^17 + 30*X^16 + 38*X^15 + 42*X^14 + 41*X^13 + 46*X^12 + 55*X^11 + 21*X^10 + 42*X^9 + 3*X^8 + X^7 + 41*X^6 + 20*X^5 + 68*X^4 + 68*X^3 + 26*X^2 + 58
key: 62
24*X^69 + 6*X^68 + 10*X^67 + 31*X^66 + 43*X^65 + 23*X^64 + 33*X^63 + 17*X^62 + 38*X^61 + 49*X^60 + 42*X^59 + 15*X^58 + 12*X^57 + 7*X^56 + 69*X^55 + 41*X^54 + 14*X^53 + 29*X^52 + 32*X^51 + 20*X^50 + 58*X^49 + 31*X^48 + 65*X^47 + 60*X^46 + 56*X^4

22*X^69 + 4*X^68 + 7*X^67 + 23*X^66 + 53*X^65 + 21*X^64 + 42*X^63 + 67*X^62 + 26*X^61 + 13*X^60 + X^59 + 6*X^58 + 32*X^57 + 38*X^56 + 23*X^55 + 19*X^54 + 63*X^53 + 70*X^52 + 28*X^51 + 48*X^50 + 16*X^49 + 18*X^48 + 50*X^47 + 32*X^46 + 63*X^44 + 40*X^43 + 32*X^42 + 47*X^41 + 2*X^40 + 68*X^39 + 37*X^38 + 15*X^37 + 49*X^36 + 33*X^35 + 68*X^34 + 38*X^33 + 51*X^32 + 69*X^31 + 32*X^30 + 9*X^28 + 44*X^27 + 11*X^26 + 12*X^25 + 16*X^24 + 63*X^23 + 35*X^22 + 25*X^21 + 24*X^20 + 12*X^19 + 28*X^18 + 8*X^17 + 39*X^16 + 23*X^15 + 10*X^14 + 31*X^13 + 61*X^12 + 33*X^11 + 42*X^10 + 24*X^9 + 46*X^8 + 56*X^7 + 11*X^6 + 12*X^5 + 55*X^4 + 30*X^3 + 69*X^2 + 4*X + 69
key: 41
X^29 is NOT present: False, coefficient: 0
33*X^69 + 63*X^68 + 66*X^67 + 41*X^66 + 37*X^65 + 9*X^64 + 47*X^63 + 30*X^62 + 54*X^61 + 19*X^60 + 30*X^59 + 36*X^58 + 43*X^57 + 49*X^56 + 18*X^55 + 9*X^54 + 66*X^53 + 57*X^52 + 69*X^51 + 11*X^50 + 22*X^49 + 67*X^48 + 24*X^47 + 44*X^46 + 5*X^45 + X^44 + 53*X^43 + 55*X^42 + 38*X^41 + 14*X^40 + 13*

In [24]:
print(find_zero_sum_coeffs(pol))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_27025/3953230053.py", line 1, in <module>
    print(find_zero_sum_coeffs(pol))
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_27025/3473250676.py", line -1, in find_zero_sum_coeffs
  File "src/cysignals/signals.pyx", line 310, in cysignals.signals.python_check_interrupt
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py", line 1997, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3/dist-packages/IPython/core/ultratb.py", line 1112, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^

X^2461 is NOT present: False, coefficient: 0
X^2821 is NOT present: False, coefficient: 0
X^6774 is NOT present: False, coefficient: 0
X^7492 is NOT present: False, coefficient: 0
X^7966 is NOT present: False, coefficient: 0
X^8174 is NOT present: False, coefficient: 0
X^9672 is NOT present: False, coefficient: 0
X^11773 is NOT present: False, coefficient: 0
X^12047 is NOT present: False, coefficient: 0
X^30769 is NOT present: False, coefficient: 0
X^35720 is NOT present: False, coefficient: 0
X^46970 is NOT present: False, coefficient: 0
X^52346 is NOT present: False, coefficient: 0
X^53673 is NOT present: False, coefficient: 0
X^53680 is NOT present: False, coefficient: 0
X^54742 is NOT present: False, coefficient: 0
